In [1]:
from utils.data_handling import *
from utils.sensor_optimisation_gpy_sparse import *
from parameters import *
import GPy
import networkx as nx

import sklearn.decomposition
import scipy
import matplotlib.pyplot as plt
np.random.seed(101)
%matplotlib inline

from tqdm import tqdm

In [2]:
parameters['i_end'] = 988
parameters['field_name'] = "Tracer"
parameters

{'i_start': 0,
 'i_end': 988,
 'crop': None,
 'field_name': 'Tracer',
 'cov_method': 'sample'}

In [3]:
loaded = initial_load_data(parameters, recompute=False)
ref_vtu, data_df, loc_df, time_df = loaded

  0%|          | 0/1 [00:00<?, ?it/s]

### Loading files from original VTU
==> Import vtu files from 0 to 0


100%|██████████| 1/1 [00:00<00:00,  2.42it/s]


Number of Locations after cropping :  100040
==> Loading from : ../data/temp_data/cache_0_988_None/loc_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/time_Tracer.pkl
==> Loading from : ../data/temp_data/cache_0_988_None/data_Tracer.pkl


In [25]:
def cell2edges(cell):
    init = np.ones(len(cell)-1).astype(int)*cell[0]
    return list(zip(init,cell[1:]))

In [26]:
def edges_list(ref_vtu):
    edges_list = []
    for i in tqdm(range(ref_vtu[0].GetLocations().shape[0]),desc="Edges list"):
        edges_list.extend(cell2edges(ref_vtu[0].GetPointPoints(i)))
    edges_list = list(set(edges_list))
    return edges_list

In [8]:
edges_list = edges_list(ref_vtu)

Edges list: 100%|██████████| 100040/100040 [00:20<00:00, 4825.40it/s]


In [9]:
G = nx.Graph()

In [10]:
G.add_edges_from(edges_list)

In [19]:
nx.set_node_attributes(G,loc_df.Z,name='Z')

In [32]:
def create_full_graph(loc_df,ref_vtu):
    all_edges = edges_list(ref_vtu)
    G = nx.Graph()
    G.add_edges_from(all_edges)
    nx.set_node_attributes(G,loc_df.X,name='X')
    nx.set_node_attributes(G,loc_df.Y,name='Y')
    nx.set_node_attributes(G,loc_df.Z,name='Z')

    
    return G
    

In [33]:
G = create_full_graph(loc_df,ref_vtu)

Edges list: 100%|██████████| 100040/100040 [00:16<00:00, 5948.67it/s]


TypeError: node_boundary() missing 1 required positional argument: 'nbunch1'

In [35]:
G.node[0]

{'Z': 250.0, 'X': -359.684632748, 'Y': -338.123650327}